In [1]:
!pip install -e LAVIS/

Obtaining file:///home/ec2-user/SageMaker/1018-lavis-local/LAVIS
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Using cached contexttimer-0.3.3-py3-none-any.whl
  Using cached decord-0.6.0-py3-none-manylinux2010_x86_64.whl (13.6 MB)
  Using cached diffusers-0.16.0-py3-none-any.whl (934 kB)
  Obtaining dependency information for einops>=0.4.1 from https://files.pythonhosted.org/packages/29/0b/2d1c0ebfd092e25935b86509a9a817159212d82aa43d7fb07eca4eeff2c2/einops-0.7.0-py3-none-any.whl.metadata
  Using cached einops-0.7.0-py3-none-any.whl.metadata (13 kB)
  Using cached fairscale-0.4.4-py3-none-any.whl
  Using cached ftfy-6.1.1-py3-none-any.whl (53 kB)
  Using cached iopath-0.1.10-py3-none-any.whl
  Using cached omegaconf-2.3.0-py3-none-any.whl (79 kB)
  Using cached opencv_python_headless-4.5.5.64-cp36-abi3-manylinux_2_17_x86_64.

In [2]:
import transformers
transformers.__version__

'4.34.1'

In [1]:
import os
os.environ['HF_HOME'] = '/home/ec2-user/SageMaker/HFHOME/'
os.environ['HF_DATASETS_CACHE'] = '/home/ec2-user/SageMaker/HFHOME/datasets/'
os.environ['TRANSFORMERS_CACHE'] = '/home/ec2-user/SageMaker/HFHOME/models/'

import torch
from PIL import Image
import requests
from lavis.models import load_model_and_preprocess

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/diffusers/models/cross_attention.py:30: FutureWarning: Importing from cross_attention is deprecated. Please import from diffusers.models.attention_processor instead.
  deprecate(


## Load Model

In [23]:
finetuned_ckpt = 's3://sagemaker-us-east-1-633205212955/output-models/blip-caption/2023-10-19-07-30-03/20231019073/checkpoint_100.pth'
!aws s3 cp {finetuned_ckpt} ./tuned_ckpt/

download: s3://sagemaker-us-east-1-633205212955/output-models/blip-caption/2023-10-19-07-30-03/20231019073/checkpoint_100.pth to tuned_ckpt/checkpoint_100.pth


In [4]:
%%writefile LAVIS/lavis/configs/models/blip2/blip2_caption_opt2.7b.yaml

model:
  arch: caption_coco_opt2.7b
  load_finetuned: True

  finetuned: /home/ec2-user/SageMaker/1018-lavis-local/tuned_ckpt/checkpoint_100.pth

  # vit encoder
  image_size: 364
  drop_path_rate: 0
  use_grad_checkpoint: False
  vit_precision: "fp32"
  freeze_vit: False

  # Q-Former
  num_query_token: 32

  # OPT
  opt_model: "facebook/opt-2.7b"

  # generation configs
  prompt: "a photo of"


preprocess:
    vis_processor:
        train:
          name: "blip_image_train"
          image_size: 364
        eval:
          name: "blip_image_eval"
          image_size: 364
    text_processor:
        train:
          name: "blip_caption"
        eval:
          name: "blip_caption"

Overwriting LAVIS/lavis/configs/models/blip2/blip2_caption_opt2.7b.yaml


In [5]:
device = torch.device("cuda") if torch.cuda.is_available() else "cpu"

model, vis_processors, _ = load_model_and_preprocess(
    name="blip2_opt", model_type="caption_coco_opt2.7b", is_eval=True, device=device
)
vis_processors.keys()

Position interpolate from 16x16 to 26x26


dict_keys(['train', 'eval'])

## Inference

In [4]:
img_url = 'https://storage.googleapis.com/sfr-vision-language-research/LAVIS/assets/merlion.png' 
raw_image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')

In [5]:
image = vis_processors["eval"](raw_image).unsqueeze(0).to(device)
model.generate({"image": image, "prompt": "Question: which city is this? Answer:"})

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


['singapore']

In [7]:
def _cap_vqa(imgpath, promptstr=None):
    raw_image = Image.open(imgpath).convert('RGB')
    image = vis_processors["eval"](raw_image).unsqueeze(0).to(torch.device("cuda"))
    if promptstr == None:
        resp = model.generate({"image": image})
    else:
        resp = model.generate({"image": image, "prompt": f"Question: {promptstr} Answer:"})
    
    return resp

In [8]:
_cap_vqa('test-images/a.jpg')

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


['a living room with a couch, loveseat and ottoman']

In [9]:
_cap_vqa('test-images/a.jpg',"what is on the floor?")

['the floor is a rug, and the couch is on top of the rug, and the loveseat is on top of the loveseat']

In [10]:
_cap_vqa('test-images/a.jpg',"what is the material of the rug on the floor?")

['the rug is made of 100% polypropylene, which is a synthetic material that is durable, easy to clean and easy to care for.']

In [11]:
_cap_vqa('test-images/c.jpg')

['a living room with a couch and coffee table']

In [12]:
_cap_vqa('test-images/c.jpg',"what is the material of the couch?")

['it is made of wood']

In [13]:
_cap_vqa('test-images/c.jpg',"what is outside the window?")

['a living room with a sofa and coffee table']